In [14]:
# Made by NaturalStupldity

In [15]:
import os
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

In [16]:
## 1. Session preparation

In [17]:
def set_random_state(seed: int):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    tf.debugging.set_log_device_placement(False)
    config = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

def clear_session():
    tf.keras.backend.clear_session()

def preparation():
    set_random_state(seed=42)
    clear_session()

In [18]:
## 2. Data preparation

In [25]:
def load_data():
    return fashion_mnist.load_data()

def one_hot_encoding(y_train, y_val, number_of_classes: int = 10):
    return to_categorical(y_train, number_of_classes), to_categorical(y_val, number_of_classes)

def normalize_images(x_train, x_val, pixel_depth: int = 255):
    return x_train.astype('float32') / pixel_depth, x_val.astype('float32') / pixel_depth

def flatten_images(x_train, x_val):
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1] * x_val.shape[2])

    return x_train, x_val

def preprocess_data(x_train, y_train, x_val, y_val):
    x_train, x_val = normalize_images(x_train=x_train, x_val=x_val)
    y_train, y_val = one_hot_encoding(y_train=y_train, y_val=y_val)

    return x_train, y_train, x_val, y_val

def build_dataset():
    (x_train, y_train), (x_val, y_val) = load_data()

    return preprocess_data(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

In [7]:
## 3. Building the model

In [41]:
def build_perceptron(input_dimension: int = 784,
                     output_dimension: int = 10,
                     loss_function="categorical_crossentropy",
                     optimizer="adam",
                     metrics=None):
    if metrics is None:
        metrics = ["accuracy"]

    model = Sequential()
    model.add(Dense(units=input_dimension, activation="relu"))
    model.add(Dense(units=128, activation="elu"))
    model.add(Dense(units=output_dimension, activation="softmax"))
    model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)

    return model

In [42]:
## 4. Training the model

In [64]:
def train_model(model, x_train, y_train, x_val, y_val, batch_size: int, epochs: int):
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

def evaluate_model(model, x_val, y_val):
    return model.evaluate(x_val, y_val)

def train(x_train, y_train, x_val, y_val, batch_size: int = 64, epochs: int = 10, verbose: bool = True):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4,
                                         weight_decay=1e-4)

    model = build_perceptron(optimizer=optimizer)
    train_model(model=model, x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, batch_size=batch_size,epochs=epochs)
    results = evaluate_model(model=model, x_val=x_val, y_val=y_val)
    if verbose:
        print("Validation loss: {}, Validation accuracy: {}".format(results[0], results[1]))

    return results

In [65]:
## 5. Main function

In [66]:
def main():
    preparation()
    x_train, y_train, x_val, y_val = build_dataset()
    x_train, x_val = flatten_images(x_train=x_train, x_val=x_val)
    train(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, batch_size=128, epochs=30)

In [67]:
main()

2023-10-16 22:50:19.334515: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-16 22:50:19.334547: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30
  5/469 [..............................] - ETA: 6s - loss: 2.4017 - accuracy: 0.1266  

2023-10-16 22:50:20.385969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


468/469 [============================>.] - ETA: 0s - loss: 0.6692 - accuracy: 0.7752

2023-10-16 22:50:26.165690: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 7s 14ms/step - loss: 0.6688 - accuracy: 0.7754 - val_loss: 0.5244 - val_accuracy: 0.8169
Epoch 2/30
469/469 [==============================] - 6s 12ms/step - loss: 0.4611 - accuracy: 0.8399 - val_loss: 0.4764 - val_accuracy: 0.8284
Epoch 3/30
469/469 [==============================] - 5s 12ms/step - loss: 0.4197 - accuracy: 0.8544 - val_loss: 0.4369 - val_accuracy: 0.8462
Epoch 4/30
469/469 [==============================] - 5s 12ms/step - loss: 0.3971 - accuracy: 0.8609 - val_loss: 0.4134 - val_accuracy: 0.8559
Epoch 5/30
469/469 [==============================] - 5s 12ms/step - loss: 0.3779 - accuracy: 0.8669 - val_loss: 0.4338 - val_accuracy: 0.8485
Epoch 6/30
469/469 [==============================] - 6s 12ms/step - loss: 0.3644 - accuracy: 0.8713 - val_loss: 0.4015 - val_accuracy: 0.8594
Epoch 7/30
469/469 [==============================] - 5s 12ms/step - loss: 0.3534 - accuracy: 0.8745 - val_loss: 0.3908 - val_accuracy: 0.8632
Epoch 8/30